In [1]:
import sys

import datasets
import torch
from transformers import AutoTokenizer

sys.path.append("..")

from logger import get_logger

%load_ext autoreload
%autoreload 2

In [1]:
import pkg_resources
for package in pkg_resources.working_set:
    if "hydra" in package.key and "ray" in package.key:
        print(package.key, "->", package._provider.module_path)

hydra-ray-launcher -> /Users/sidbaskaran/miniconda3/envs/interp/lib/python3.10/site-packages


/var/folders/ym/vc9z3h415dn008d8y48sr2h80000gn/T/ipykernel_73597/667104175.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


In [2]:
x = torch.randn(4,5,5)
eigs = torch.linalg.eigvals(x)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
tokenizer.pad_token_id = tokenizer.eos_token_id

In [ ]:
tokenizer.convert_ids_to_tokens(
    tokenizer(
        tokenizer.bos_token + "Test sentence.",
        add_special_tokens=True,
        padding=True,
        max_length=32,
    ).input_ids
)

In [2]:
ds_train = datasets.load_from_disk(
    "/workspace/HyperDAS/experiments/RAVEL/data/city_country_train"
)
ds_test = datasets.load_from_disk(
    "/workspace/HyperDAS/experiments/RAVEL/data/city_country_test"
)

In [ ]:
ds_test[0]

In [ ]:
ds_train, ds_test

In [ ]:
sd = torch.load(
    "/workspace/HyperDAS/assets/checkpoints/ReflectSelect_20241029_161135/final_model/das.pt"
)

In [ ]:
for k in sd.keys():
    print(k)

In [ ]:
sd["rotate_layer.parametrizations.weight.original"].shape

In [ ]:
sd["rotate_layer.parametrizations.weight.0.base"].shape

In [7]:
orth_linear = torch.nn.utils.parametrizations.orthogonal(torch.nn.Linear(20, 40))

In [9]:
sdt = orth_linear.state_dict()

In [ ]:
(
    sdt["parametrizations.weight.0.base"].shape,
    sdt["parametrizations.weight.original"].shape,
)

In [ ]:
torch.dist(
    sdt["parametrizations.weight.original"].T @ sdt["parametrizations.weight.original"],
    torch.eye(20),
)

In [ ]:
logger = get_logger(__name__)
logger.debug("This is a debug message")

In [ ]:
x = torch.nn.Parameter(torch.randn(5, 5), requires_grad=True)

y = 5 * x + 2

vals, indices = torch.topk(y, 2)

print(vals.requires_grad, indices.requires_grad)

loss = vals.norm()
loss.backward()

In [ ]:
vals._grad

In [ ]:
import torch
import torch.nn as nn


class TopKSTE(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, k):
        # Get the values and indices of the top k elements
        vals, indices = torch.topk(input, k, dim=-1)
        ctx.save_for_backward(indices, torch.tensor(input.shape))
        return vals, indices

    @staticmethod
    def backward(ctx, grad_output, grad_indices):
        indices, input_shape = ctx.saved_tensors
        grad_input = torch.zeros(tuple(input_shape), device=grad_output.device)
        grad_input.scatter_(-1, indices, grad_output)
        return grad_input, None


# Test function
def test_topk_ste():
    # Create a simple model that uses TopKSTE
    class SimpleModel(nn.Module):
        def __init__(self):
            super().__init__()
            self.linear = nn.Linear(5, 5)
            self.topk_ste = TopKSTE.apply

        def forward(self, x):
            x = self.linear(x)
            values, indices = self.topk_ste(x, 3)
            return values.sum(), indices

    # Create an instance of the model
    model = SimpleModel()

    # Create a random input tensor
    x = torch.randn(1, 5, requires_grad=True)

    # Forward pass
    output, indices = model(x)

    # Backward pass
    output.backward()

    # Print results
    print("Input:", x)
    print("Output (sum of top-3 values):", output)
    print("Top-3 indices:", indices)
    print("Input gradient:", x.grad)
    print("Linear layer weight gradient:", model.linear.weight.grad)


# Run the test
test_topk_ste()